# Projeto Aprendizagem automática

<hr style="border:2px solid black"> </hr>

# Imports

In [1]:
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
from sklearn.linear_model import RidgeClassifier
from sklearn.decomposition import TruncatedSVD

<hr style="border:2px solid black"> </hr>

# Funções para competição 

In [2]:
def text2vector(Docs):
	
	dictionary =pickle.load(open('A45102_classificador.p','rb'))

	docs = clear_data(Docs,False)

	docs = stemming(Docs,3)

	tf_idf = dictionary['tf-idf']

	return tf_idf.transform(docs)


def binClassify(X):

	dictionary =pickle.load(open('A45102_classificador.p','rb'))

	logReg = dictionary['bin']

	return logReg.predict(X)


def multiClassify(X):

	dictionary =pickle.load(open('A45102_classificador.p','rb'))

	logReg = dictionary['multi']

	return logReg.predict(X)

### Funções contrução do ficheiro pickle 

In [3]:
#função para criar um ficheiro pickle com o objeto tf-idf
def builder_td_idf(Docs):

	dictionary = {}

	x_train = clear_data(Docs,False)

	x_train = stemming(Docs,3)

	tf_idf = TfidfVectorizer(min_df=3,token_pattern=r'\b\w+\b',ngram_range=(1,2)).fit(Docs)

	dictionary['tf-idf'] = tf_idf

	pickle.dump(dictionary,open('A45102_classificador.p','wb'))


#função para criar objeto classificador binário
def builder_bin_classify(X,y):

	dictionary =pickle.load(open('A45102_classificador.p','rb'))

	print(dictionary.keys())

	logReg = LogisticRegression(C=2,multi_class='multinomial',max_iter=1000,penalty='l2',solver='saga').fit(X,y)

	dictionary['bin'] = logReg

	pickle.dump(dictionary,open('A45102_classificador.p','wb'))


#função para criar objeto classificador multi-classe
def builder_multi_classify(X,y):

	dictionary =pickle.load(open('A45102_classificador.p','rb'))

	print(dictionary.keys())

	logReg = LogisticRegression(C=1.9,multi_class='ovr',max_iter=1000,penalty='l1',solver='saga').fit(X,y)

	dictionary['multi'] = logReg

	pickle.dump(dictionary,open('A45102_classificador.p','wb'))

<hr style="border:2px solid black"> </hr>

# Funções conversão

### Função de conversão dos indices para binário

In [4]:
#função para converter os indices de 1-4 para 0 e os indices 7-10 para 1 (multi-classe para binário)
#@args 
#-> target pontuação das críticas
#@return array em formato binário
def convert_indexes_into_binary(target):

	size = len(target)

	data_converted = np.zeros(size).astype('int8')

	for i in range(size):
		if target[i] > 4:
			data_converted[i] = 1

	return data_converted

### Função de limpeza dos textos

In [5]:
#função para remover caracteres que não constituem muita importância para a classificação
#@args 
#-> docs são os dados
#-> str_bytes é uma flag || true -> dados estão em bytes, false -> str unicode do python
#@return dados limpos
def clear_data(docs,str_bytes):
	if str_bytes:
		#converter codificação dos caracteres bytes ASCII para UTF-8
		data = [doc.decode('UTF-8') for doc in docs]

	#tirar os caracteres mudança linha em HTML por espaço
	docs = [doc.replace('<br />',' ') for doc in docs]

	#remover todos os caraceteres não alfabeticos e deixa os caracteres com acentos tipo é e ç
	#feito através de uma expressão regular
	docs = [re.sub(r'[^a-zA-Z0-9\u00C0-\u00FF]+',' ',doc) for doc in docs]
	return docs

### Função de stemming do texto

In [6]:
#função que converte várias palavras relacionadas numa única
#permite reduzir o tamanho da lista dos tokens
#@args
#-> docs são os dados
#-> type_stemming (inteiro) tipo de stemming 1 - Porter Stemmer, 2 -> Lancaster Stemmer, outro -> Snowball Stemmer
#@return dados com as palavras relacionadas convertidas convertidas numa unica palavra 
def stemming(docs,type_stemming):
	if type_stemming == 1:
		#criação do objeto Porter Stemmer
		stemmer_porter = PorterStemmer()
		#metodo join permite juntar strings depois fazer o stemming
		#tem de fazer isto a todas as palavras em cada documento por sua vez em todos os documentos
		stemming_porter = [' '.join([stemmer_porter.stem(word) for word in doc.split()]) for doc in docs]
		return stemming_porter
	
	elif type_stemming == 2:
		stemmer_lancaster = LancasterStemmer()
		stemming_lancaster = [' '.join([stemmer_lancaster.stem(word) for word in doc.split()]) for doc in docs]
		return stemming_lancaster

	else:
		stemmer_snowball = SnowballStemmer(language='english')
		stemming_snowball = [' '.join([stemmer_snowball.stem(word) for word in doc.split()]) for doc in docs]
		return stemming_snowball

### Função converção dos textos na representação tf-idf

In [7]:
#função para converte textos de strings em tokens 
#@args 
#-> docs são os dados
#-> min_df_value (inteiro) número de vezes que a palavra deve aparecer em x corpus
#-> num_characters (inteiro) tamanho minimo da palavra
#-> n_grams (inteiro) para criar bi-grama, tri-gramas e quadri-gramas
#@return objeto do tipo tf-idf
def convert_str_to_tf_idf(docs, min_df_value,num_characters,n_grams):
	#converte o valor inteiro na expressão do token_pattern
	pattern = r'\b'
	for i in range(num_characters):
		pattern = pattern + '\w'
	pattern = pattern + '+'+r'\b'
	

	if n_grams == 1:
		#também pode ser feito CountVectorizer() e TfidfTransformer()
		#min_df -> escolhe só palavras que aparecam em pelo menos x (o valor do argumento) corpus  
		#token_pattern -> caracteres que tenham quatro ou mais caracteres
		return TfidfVectorizer(min_df=min_df_value,token_pattern=pattern).fit(docs)

	elif n_grams == 2 or n_grams == 3 or n_grams == 4:

		return TfidfVectorizer(min_df=min_df_value,token_pattern=pattern,ngram_range=(1,n_grams)).fit(docs)

### Função para converter os dados numa matriz esparsa 

In [8]:
#função para converte textos de strings em tokens 
#@args 
#-> tf_idf objeto do tipo td-idf
#-> docs dados
#@return matriz esparsa to tf-idf
def convert_to_sparce_matrix(tf_idf, docs):

	return tf_idf.transform(docs)

<hr style="border:2px solid black"> </hr>

# Código para testar os classificadores para novos dados 

In [ ]:
dictionary = pickle.load(open(None,'rb'))#alterar o valor None para o nome do ficheiro


#fazer load dos dados
Docs = dictionary[None] #ir buscar a key para os dados


#ir buscar a classificação correspondente
y = dictionary[None] #ir buscar a key para a classificação


#variável com os indices 0-1
y0 = convert_indexes_into_binary(y)


#conversão do texto para 
x_test = text2vector(Docs)


# resultado da classificação multi-classe
y_test_multi = multiClassify(x_test)


# resultado da classificação binário
y_test_bin = binClassify(x_test)


<hr style="border:2px solid black"> </hr>

# Funções de teste

### Funções complementares (necessárias para as de teste) 

In [9]:
#retorna o tipo de penalização do discriminante logistico
def __get_penalty(value):

	if value == 1:
		return 'l1'

	elif value == 2:
		return 'elasticnet'

	else:
		return 'l2'


#função que retorna o valor do class_weight do discriminante logistico
def __get_balance_value(flag):

	if flag:
		return 'balanced'

	else:
		return None

#função que retorna o valor da multi classe do discriminante logistico
def __get_multi_class(value):

	if value == 1:
		return 'auto'

	elif value == 2:
		return 'multinomial'

	else:
		return 'ovr'

	
#retorna o tipo de solver do discriminante logistico
def __get_solver(value,penalty):

	
	if value == 1 and penalty != 1 and penalty != 2:
		return 'lbfgs'

	elif value == 2 and penalty != 1 and penalty != 2:
		return 'sag'

	elif value == 3 and penalty != 1 and penalty != 2:
		return 'newton-cg'

	else:
		return 'saga'

### Função de teste dos parâmetros do discriminante logistico 

In [10]:
#função imprime na consola os scores para vários parâmetros do discriminante logistico
#@args
#-> data_train -> dados treino após ser feito o transform do tf-idf
#-> y_train -> true class do treino 
#-> data_test -> dados teste após ser feito o transform do tf-idf
#-> y_test -> true class do test
#-> penalty -> 1 -> auto, 2 -> multinomial, outro -> ovr
#-> solver -> 1 -> lbfgs, 2 -> sag, 3 -> sag, 4 -> newton-cg, outro -> saga
#-> c -> valor para a regidez da penalização
#-> balance -> True para classificar baseado nos pesos das classes
#-> multi_class -> 1 -> auto, 2 -> multinomial, outro -> ovr
def test_logistic_regression(data_train, y_train, data_test, y_test, penalty, solver,max_iter, c, balance, multi_class):


	print("penalização: "+__get_penalty(penalty)+", resolver: "+__get_solver(solver,penalty)+", valor de C: "+str(c)+", balanceamento: "+str(__get_balance_value(balance))+", multi-classe: "+__get_multi_class(multi_class))
	
	if penalty == 2:
		logReg = LogisticRegression(penalty=__get_penalty(penalty),solver=__get_solver(solver,penalty),max_iter=1000,
			C=c,class_weight=__get_balance_value(balance),multi_class=__get_multi_class(multi_class),l1_ratio=0.5)

	else:
		logReg = LogisticRegression(penalty=__get_penalty(penalty),solver=__get_solver(solver,penalty),max_iter=1000,
			C=c,class_weight=__get_balance_value(balance),multi_class=__get_multi_class(multi_class))


	logReg.fit(data_train,y_train)
	print(round(logReg.score(data_train,y_train),4))
	print(round(logReg.score(data_test,y_test),4),"\n")
    
    
#função imprime na consola os scores e os coeficientes utilizados para vários parâmetros do discriminante logistico
def test_logistic_regression_coef(data_train, y_train, data_test, y_test, penalty, solver,max_iter, c, balance, multi_class):


	print("penalização: "+__get_penalty(penalty)+", resolver: "+__get_solver(solver,penalty)+", valor de C: "+str(c)+", balanceamento: "+str(__get_balance_value(balance))+", multi-classe: "+__get_multi_class(multi_class))
	

	if penalty == 2:
		logReg = LogisticRegression(penalty=__get_penalty(penalty),solver=__get_solver(solver,penalty),max_iter=1000,
			C=c,class_weight=__get_balance_value(balance),multi_class=__get_multi_class(multi_class),l1_ratio=0.5)

	else:
		logReg = LogisticRegression(penalty=__get_penalty(penalty),solver=__get_solver(solver,penalty),max_iter=1000,
			C=c,class_weight=__get_balance_value(balance),multi_class=__get_multi_class(multi_class))


	logReg.fit(data_train,y_train)

	print(round(logReg.score(data_train,y_train),4))
	print(round(logReg.score(data_test,y_test),4))
	print("Coeficientes utilizados: "+str(np.sum(logReg.coef_.squeeze() != 0)),"\n")

### Função de teste do classificador Ridge 

In [11]:
#função para testar os parâmetros do classificador ridge
#@args
#-> data_train -> dados treino após ser feito o transform do tf-idf
#-> y_train -> true class do treino 
#-> data_test -> dados teste após ser feito o transform do tf-idf
#-> y_test -> true class do test
#-> alpha -> força da regularização
#-> fit_intercept -> Se for falso não será utilizado interceção do modelo nos calculos
#-> normalize -> Se for True o regressor é normalizado antes de ser calculado, por subtração da média pela norma l2
def test_ridge(data_train, y_train, data_test, y_test, alpha, fit_intercept,normalize):


	print("Alpha: "+str(alpha)+", fit_intercept: "+str(fit_intercept)+", normalize: "+str(normalize))
	

	linReg = RidgeClassifier(alpha=alpha,fit_intercept=fit_intercept,normalize=normalize)


	linReg.fit(data_train,y_train)

	print("score treino:",round(linReg.score(data_train,y_train),4))
	print("score teste:",round(linReg.score(data_test,y_test),4))
	print("Coeficientes utilizados: "+str(np.sum(linReg.coef_.squeeze() != 0)),"\n")

<hr style="border:2px solid black"> </hr>

# Testes realizados

## Testes para classificação multiclasse 

### Carregar os dados

In [10]:
#data é as criticas
#target o resultado das classificações
dictionary =pickle.load(open('imdbCriticas.p','rb'))


#os dados já vêm em strings unicode
X = dictionary.data

#valores estão baralhados
y = dictionary.target

#divisão em treino e teste com uma aleatoridade sempre igual(para os resultados serem sempre constantes)
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=1/4,shuffle=True,random_state=0)


### Limpeza dos dados de treino e teste

In [11]:
#2º argumento -> true caso os dados seja uma string de bytes
x_train = clear_data(x_train,False)

x_test = clear_data(x_test,False)

### Stemming 

In [12]:
#2º argumento -> 1 - Porter Stemmer, 2 -> Lancaster Stemmer, outro -> Snowball Stemmer
x_train = stemming(x_train,3)

x_test = stemming(x_test,3)

### Representação em tf-idf

In [15]:
#2º argumento -> tamanho minimo da palavra
#3º argumento -> número de vezes minima palavra aparecer
#4º argumento -> n-gramas (só funciona normal e bi-grama)
tf_idf = convert_str_to_tf_idf(x_train,3,1,1)


#1º argumento -> instância de td-idf
#2º argumento -> dados
data_train = convert_to_sparce_matrix(tf_idf, x_train)

data_test = convert_to_sparce_matrix(tf_idf, x_test)

### Teste para diferentes penalizações e valores de C

In [16]:
#valor de c para testar a classificação
c_values = [.5,1,1.5,2]


for i in range(len(c_values)):

	c = c_values[i]

	#testar a função para l1, solver = saga, class_weight = None, multi_class = multinomial
	test_logistic_regression_coef(data_train,y_train,data_test,y_test,1,0,1000,c,False,2)

	#testar a função para elasticinet, l1_ratio=0.5, solver = saga, class_weight = None, multi_class = multinomial
	test_logistic_regression_coef(data_train,y_train,data_test,y_test,2,0,1000,c,False,2)

	#testar a função para l2, solver = saga, class_weight = None, multi_class = multinomial
	test_logistic_regression_coef(data_train,y_train,data_test,y_test,3,0,1000,c,False,2)

penalização: l1, resolver: saga, valor de C: 0.5, balanceamento: None, multi-classe: multinomial
0.4815
0.4473
Coeficientes utilizados: 1718 

penalização: elasticnet, resolver: saga, valor de C: 0.5, balanceamento: None, multi-classe: multinomial
0.4967
0.4515
Coeficientes utilizados: 5921 

penalização: l2, resolver: saga, valor de C: 0.5, balanceamento: None, multi-classe: multinomial
0.5953
0.4583
Coeficientes utilizados: 193752 

penalização: l1, resolver: saga, valor de C: 1, balanceamento: None, multi-classe: multinomial
0.5288
0.4524
Coeficientes utilizados: 4414 

penalização: elasticnet, resolver: saga, valor de C: 1, balanceamento: None, multi-classe: multinomial
0.5605
0.458
Coeficientes utilizados: 14974 

penalização: l2, resolver: saga, valor de C: 1, balanceamento: None, multi-classe: multinomial
0.67
0.4585
Coeficientes utilizados: 193752 

penalização: l1, resolver: saga, valor de C: 1.5, balanceamento: None, multi-classe: multinomial
0.5816
0.4484
Coeficientes utiliz

### Teste para balançeamento das classes

In [17]:
c_values = [.1,.5,1,2]

for i in range(len(c_values)):

	c = c_values[i]

	#testar o parâmetro class_weight para None
	test_logistic_regression(data_train,y_train,data_test,y_test,1,0,1000,c,False,2)

	#testar o parâmetro class_weight para 'balanced'
	test_logistic_regression(data_train,y_train,data_test,y_test,1,0,1000,c,True,2)

penalização: l1, resolver: saga, valor de C: 0.1, balanceamento: None, multi-classe: multinomial
0.4017
0.3987 

penalização: l1, resolver: saga, valor de C: 0.1, balanceamento: balanced, multi-classe: multinomial
0.3751
0.3644 

penalização: l1, resolver: saga, valor de C: 0.5, balanceamento: None, multi-classe: multinomial
0.4816
0.4475 

penalização: l1, resolver: saga, valor de C: 0.5, balanceamento: balanced, multi-classe: multinomial
0.4623
0.4122 

penalização: l1, resolver: saga, valor de C: 1, balanceamento: None, multi-classe: multinomial
0.5288
0.4523 

penalização: l1, resolver: saga, valor de C: 1, balanceamento: balanced, multi-classe: multinomial
0.5231
0.4164 

penalização: l1, resolver: saga, valor de C: 2, balanceamento: None, multi-classe: multinomial
0.6322
0.4416 

penalização: l1, resolver: saga, valor de C: 2, balanceamento: balanced, multi-classe: multinomial
0.6418
0.4062 



###  Teste para o parâmetro multi-classe

In [18]:
for i in range(len(c_values)):

	c = c_values[i]

	#testar o parâmetro multi_class para 'auto'
	test_logistic_regression(data_train,y_train,data_test,y_test,1,0,1000,c,False,1)

	#testar o parâmetro multi_class para 'multinomial'
	test_logistic_regression(data_train,y_train,data_test,y_test,1,0,1000,c,False,2)

	#testar o parâmetro multi_class para 'ovr'
	test_logistic_regression(data_train,y_train,data_test,y_test,1,0,1000,c,False,3)

penalização: l1, resolver: saga, valor de C: 0.1, balanceamento: None, multi-classe: auto
0.4018
0.3987 

penalização: l1, resolver: saga, valor de C: 0.1, balanceamento: None, multi-classe: multinomial
0.4017
0.3987 

penalização: l1, resolver: saga, valor de C: 0.1, balanceamento: None, multi-classe: ovr
0.3961
0.3914 

penalização: l1, resolver: saga, valor de C: 0.5, balanceamento: None, multi-classe: auto
0.4816
0.4474 

penalização: l1, resolver: saga, valor de C: 0.5, balanceamento: None, multi-classe: multinomial
0.4817
0.4475 

penalização: l1, resolver: saga, valor de C: 0.5, balanceamento: None, multi-classe: ovr
0.4776
0.4446 

penalização: l1, resolver: saga, valor de C: 1, balanceamento: None, multi-classe: auto
0.5289
0.4524 

penalização: l1, resolver: saga, valor de C: 1, balanceamento: None, multi-classe: multinomial
0.5288
0.4524 

penalização: l1, resolver: saga, valor de C: 1, balanceamento: None, multi-classe: ovr
0.5237
0.451 

penalização: l1, resolver: saga, va

### Teste para o parâmetro maximo iterações

In [19]:
#testar os diferentes valores de máxima iterações
for i in range(len(c_values)):

	c = c_values[i]

	#testar o parâmetro max_iter para 1000
	test_logistic_regression(data_train,y_train,data_test,y_test,1,0,1000,c,False,2)

	#testar o parâmetro max_iter para 10000
	test_logistic_regression(data_train,y_train,data_test,y_test,1,0,10000,c,False,2)

penalização: l1, resolver: saga, valor de C: 0.1, balanceamento: None, multi-classe: multinomial
0.4017
0.3986 

penalização: l1, resolver: saga, valor de C: 0.1, balanceamento: None, multi-classe: multinomial
0.4018
0.3987 

penalização: l1, resolver: saga, valor de C: 0.5, balanceamento: None, multi-classe: multinomial
0.4816
0.4475 

penalização: l1, resolver: saga, valor de C: 0.5, balanceamento: None, multi-classe: multinomial
0.4817
0.4473 

penalização: l1, resolver: saga, valor de C: 1, balanceamento: None, multi-classe: multinomial
0.5289
0.4524 

penalização: l1, resolver: saga, valor de C: 1, balanceamento: None, multi-classe: multinomial
0.5288
0.4524 

penalização: l1, resolver: saga, valor de C: 2, balanceamento: None, multi-classe: multinomial
0.6322
0.4416 

penalização: l1, resolver: saga, valor de C: 2, balanceamento: None, multi-classe: multinomial
0.6322
0.4416 



### Teste para diferentes stemmings para diferentes valores de C

In [22]:
for i in range(len(c_values)):

	print("valor de C:",c_values[i])

	for x in range(1,4):

		print("stemming: "+str(x))

		#2º argumento -> 1 - Porter Stemmer, 2 -> Lancaster Stemmer, outro -> Snowball Stemmer
		x_train = stemming(x_train,x)

		x_test = stemming(x_test,x)


		tf_idf = convert_str_to_tf_idf(x_train,3,1,1)


		#1º argumento -> instância de td-idf
		#2º argumento -> dados
		data_train = convert_to_sparce_matrix(tf_idf, x_train)

		data_test = convert_to_sparce_matrix(tf_idf, x_test)


		logReg = LogisticRegression(penalty='l1',solver='saga',max_iter=1000,C=c_values[i],tol=1e-3,multi_class='multinomial')
		logReg.fit(data_train,y_train)


		print(logReg.score(data_train,y_train))
		print(logReg.score(data_test,y_test))

valor de C: 0.1
stemming: 1
0.4022
0.3988
stemming: 2
0.40323333333333333
0.3987
stemming: 3
0.40323333333333333
0.3987
valor de C: 0.5
stemming: 1
0.48036666666666666
0.4469
stemming: 2
0.47763333333333335
0.4471
stemming: 3
0.4779
0.4471
valor de C: 1
stemming: 1
0.5284666666666666
0.4497
stemming: 2
0.529
0.4498
stemming: 3
0.529
0.4501
valor de C: 2
stemming: 1
0.6198333333333333
0.4436
stemming: 2
0.6194333333333333
0.4437
stemming: 3
0.6195
0.4438


###  Teste para os parâmetros do tf-idf e tamanho dos tokens para n-gramas = 2

In [28]:
for sizeWord in range(1,6):

	for appearTexts in range(1,6):

		print("Tamanho da palavra:",sizeWord)

		print("Quantas vezes aparece a palavra:",appearTexts)


		tf_idf = convert_str_to_tf_idf(x_train,sizeWord,appearTexts,2)
		tokens = tf_idf.get_feature_names()
		print(len(tokens))


		#1º argumento -> instância de td-idf
		#2º argumento -> dados
		data_train = convert_to_sparce_matrix(tf_idf, x_train)

		data_test = convert_to_sparce_matrix(tf_idf, x_test)

		logReg = LogisticRegression(penalty='l1',solver='saga',max_iter=1000,C=1,tol=1e-3,multi_class='multinomial')
		logReg.fit(data_train,y_train)


		print(logReg.score(data_train,y_train))
		print(logReg.score(data_test,y_test),"\n")

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 1
1277898
0.5033333333333333
0.4684 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 2
1329250
0.46236666666666665
0.4307 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 3
1428136
0.45876666666666666
0.427 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 4
1495784
0.4514
0.4196 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 5
1169413
0.4337
0.3933 

Tamanho da palavra: 2
Quantas vezes aparece a palavra: 1
407593
0.5101333333333333
0.4706 

Tamanho da palavra: 2
Quantas vezes aparece a palavra: 2
414140
0.4723
0.4299 

Tamanho da palavra: 2
Quantas vezes aparece a palavra: 3
405353
0.4688
0.4289 

Tamanho da palavra: 2
Quantas vezes aparece a palavra: 4
374830
0.4653333333333333
0.4205 

Tamanho da palavra: 2
Quantas vezes aparece a palavra: 5
247655
0.4509666666666667
0.399 

Tamanho da palavra: 3
Quantas vezes aparece a palavra: 1
254128
0.5136
0.4711 

Tamanho da palavra: 3
Quantas vezes aparece

###  Teste para os parâmetros do tf-idf e tamanho dos tokens para n-gramas = 1

In [29]:
for sizeWord in range(1,6):

	for appearTexts in range(1,6):

		print("Tamanho da palavra:",sizeWord)

		print("Quantas vezes aparece a palavra:",appearTexts)


		tf_idf = convert_str_to_tf_idf(x_train,sizeWord,appearTexts,1)
		tokens = tf_idf.get_feature_names()
		print(len(tokens))


		#1º argumento -> instância de td-idf
		#2º argumento -> dados
		data_train = convert_to_sparce_matrix(tf_idf, x_train)

		data_test = convert_to_sparce_matrix(tf_idf, x_test)

		logReg = LogisticRegression(penalty='l1',solver='saga',max_iter=1000,C=1,tol=1e-3,multi_class='multinomial')
		logReg.fit(data_train,y_train)


		print(logReg.score(data_train,y_train))
		print(logReg.score(data_test,y_test),"\n")

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 1
53673
0.5262
0.4518 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 2
53632
0.5041666666666667
0.4269 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 3
52967
0.5051
0.426 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 4
50359
0.5032666666666666
0.4174 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 5
44277
0.48793333333333333
0.4054 

Tamanho da palavra: 2
Quantas vezes aparece a palavra: 1
30486
0.5280666666666667
0.4519 

Tamanho da palavra: 2
Quantas vezes aparece a palavra: 2
30447
0.5062
0.4272 

Tamanho da palavra: 2
Quantas vezes aparece a palavra: 3
29932
0.5073666666666666
0.4267 

Tamanho da palavra: 2
Quantas vezes aparece a palavra: 4
28280
0.5053333333333333
0.4185 

Tamanho da palavra: 2
Quantas vezes aparece a palavra: 5
24280
0.4916
0.4069 

Tamanho da palavra: 3
Quantas vezes aparece a palavra: 1
24219
0.5287666666666667
0.4523 

Tamanho da palavra: 3
Quantas vezes aparece a p

### Teste para saber quais os melhores parâmetros do discriminante logistico

In [34]:
penaltyTest = ['l1','l2']
multiClass = ['multinomial','ovr','auto']
cList = [0.1,0.5,1,1.1,1.2,1.3,1.4,1.5,2]
grelha={'multi_class':multiClass,'C':cList,'penalty':penaltyTest}

gSearch = GridSearchCV(LogisticRegression(solver='saga',max_iter=1000),param_grid=grelha,cv=2,verbose=1,n_jobs=3)
gSearch.fit(data_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(data_train,y_train))
print('acertos no teste: ',svm.score(data_test,y_test))
print('Parâmetros multi-classe:',par['multi_class'],'C:',par['C'],'penalty:',par['penalty'])

Fitting 2 folds for each of 54 candidates, totalling 108 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  2.4min
[Parallel(n_jobs=3)]: Done 108 out of 108 | elapsed: 13.4min finished


acertos no treino:  0.5324666666666666
acertos no teste:  0.4536
Parâmetros multi-classe: ovr C: 1.1 penalty: l1


<hr style="border:2px solid black"> </hr>

## Teste para os melhores parâmetros para uma dada semente 

### Teste sobre os melhores parâmetros de tf-idf e discriminante logistico 

In [38]:
logReg = LogisticRegression(solver='saga',max_iter=1000)

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('logistic', logReg)])

param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w\w+\b', r'\b\w\w\w\w\w+\b'],
    'tfidf__ngram_range': [(1,1), (1,2)],
    'logistic__penalty': ['l1','l2'],
    'logistic__C': [0.5,1,1.5,2],
    'logistic__multi_class': ['multinomial','ovr']
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 192 candidates, totalling 384 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  5.6min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 32.9min
[Parallel(n_jobs=3)]: Done 384 out of 384 | elapsed: 105.0min finished


acertos no treino:  0.5958666666666667
acertos no teste:  0.4822
Parâmetros: {'logistic__C': 2, 'logistic__multi_class': 'ovr', 'logistic__penalty': 'l1', 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


In [21]:
logReg = LogisticRegression(solver='saga',max_iter=1000)

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('logistic', logReg)])

param_grid = {
    'tfidf__min_df': [1, 3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'logistic__penalty': ['l1','l2'],
    'logistic__C': [0.5,1,1.1,1.2,1.5,2],
    'logistic__multi_class': ['multinomial','ovr']
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 144 candidates, totalling 288 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 18.3min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 111.1min
[Parallel(n_jobs=3)]: Done 288 out of 288 | elapsed: 191.7min finished


acertos no treino:  0.5958666666666667
acertos no teste:  0.4822
Parâmetros: {'logistic__C': 2, 'logistic__multi_class': 'ovr', 'logistic__penalty': 'l1', 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


In [22]:
logReg = LogisticRegression(solver='saga',max_iter=1000)

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('logistic', logReg)])

param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'logistic__penalty': ['l1','l2'],
    'logistic__C': [1.9,2,2.1,2.2],
    'logistic__multi_class': ['multinomial','ovr']
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 32 candidates, totalling 64 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 37.8min
[Parallel(n_jobs=3)]: Done  64 out of  64 | elapsed: 54.3min finished


acertos no treino:  0.5863666666666667
acertos no teste:  0.4829
Parâmetros: {'logistic__C': 1.9, 'logistic__multi_class': 'ovr', 'logistic__penalty': 'l1', 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


In [13]:
logReg = LogisticRegression(solver='saga',max_iter=1000)

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('logistic', logReg)])

param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'logistic__penalty': ['l1','l2'],
    'logistic__C': [1.6,1.7,1.8,1.9],
    'logistic__multi_class': ['multinomial','ovr']
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 32 candidates, totalling 64 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 32.0min
[Parallel(n_jobs=3)]: Done  64 out of  64 | elapsed: 47.0min finished


acertos no treino:  0.5864
acertos no teste:  0.4829
Parâmetros: {'logistic__C': 1.9, 'logistic__multi_class': 'ovr', 'logistic__penalty': 'l1', 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


<hr style="border:2px solid black"> </hr>

## Testes para sementes aleatórias 

In [12]:
dictionary =pickle.load(open('imdbCriticas.p','rb'))

X = dictionary.data
y = dictionary.target

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=1/4,shuffle=True)

x_train = clear_data(x_train,False)

x_test = clear_data(x_test,False)

x_train = stemming(x_train,3)

x_test = stemming(x_test,3)

In [11]:
logReg = LogisticRegression(solver='saga',max_iter=1000)

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('logistic', logReg)])

param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'logistic__penalty': ['l1','l2'],
    'logistic__C': [1.8,1.9,2],
    'logistic__multi_class': ['multinomial','ovr']
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  48 out of  48 | elapsed: 38.9min finished


acertos no treino:  0.5974333333333334
acertos no teste:  0.4782
Parâmetros: {'logistic__C': 1.9, 'logistic__multi_class': 'ovr', 'logistic__penalty': 'l1', 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


In [13]:
logReg = LogisticRegression(solver='saga',max_iter=1000)

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('logistic', logReg)])

param_grid = {
    'tfidf__min_df': [1, 3, 5],
    'tfidf__token_pattern': [r'\b\w+\b',r'\b\w\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'logistic__penalty': ['l1','l2'],
    'logistic__C': [1.9,2],
    'logistic__multi_class': ['multinomial','ovr']
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 44.0min
[Parallel(n_jobs=3)]: Done  96 out of  96 | elapsed: 91.4min finished


acertos no treino:  0.5952
acertos no teste:  0.4835
Parâmetros: {'logistic__C': 2, 'logistic__multi_class': 'ovr', 'logistic__penalty': 'l1', 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


<hr style="border:2px solid black"> </hr>

## Testes para classificação binária 

In [14]:
dictionary =pickle.load(open('imdbCriticas.p','rb'))

X = dictionary.data
y = dictionary.target

y = convert_indexes_into_binary(y)

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=1/4,shuffle=True,random_state=0)

In [15]:
#2º argumento -> true caso os dados seja uma string de bytes
x_train = clear_data(x_train,False)

x_test = clear_data(x_test,False)


#2º argumento -> 1 - Porter Stemmer, 2 -> Lancaster Stemmer, outro -> Snowball Stemmer
x_train = stemming(x_train,3)

x_test = stemming(x_test,3)

In [41]:
#Para os diferentes valores de min_df, token_pattern para n-grams = 1
for i in range(1,6):

	for j in range(1,6):

		print("Valor tamanho minimo:",i)

		print("Aparecimento dos textos:",j)

		tf_idf = convert_str_to_tf_idf(x_train,i,j,1)
		tokens = tf_idf.get_feature_names()
		print("Tamanho dos tokens: ",len(tokens))

		data_train = convert_to_sparce_matrix(tf_idf, x_train)

		data_test = convert_to_sparce_matrix(tf_idf, x_test)


		logReg = LogisticRegression(penalty='l1',solver='saga',max_iter=1000,C=1,tol=1e-3,multi_class='ovr')
		logReg.fit(data_train,y_train)

		print(logReg.score(data_train,y_train))
		print(logReg.score(data_test,y_test))

Valor tamanho minimo: 1
Aparecimento dos textos: 1
Tamanho dos tokens:  53673
0.9029333333333334
0.8819
Valor tamanho minimo: 1
Aparecimento dos textos: 2
Tamanho dos tokens:  53632
0.9004333333333333
0.8829
Valor tamanho minimo: 1
Aparecimento dos textos: 3
Tamanho dos tokens:  52967
0.8979333333333334
0.8816
Valor tamanho minimo: 1
Aparecimento dos textos: 4
Tamanho dos tokens:  50359
0.8951666666666667
0.8723
Valor tamanho minimo: 1
Aparecimento dos textos: 5
Tamanho dos tokens:  44277
0.8784333333333333
0.854
Valor tamanho minimo: 2
Aparecimento dos textos: 1
Tamanho dos tokens:  30486
0.9031333333333333
0.8818
Valor tamanho minimo: 2
Aparecimento dos textos: 2
Tamanho dos tokens:  30447
0.9008333333333334
0.8825
Valor tamanho minimo: 2
Aparecimento dos textos: 3
Tamanho dos tokens:  29932
0.8991333333333333
0.8811
Valor tamanho minimo: 2
Aparecimento dos textos: 4
Tamanho dos tokens:  28280
0.8954333333333333
0.8728
Valor tamanho minimo: 2
Aparecimento dos textos: 5
Tamanho dos to

<hr style="border:2px solid black"> </hr>

## Teste melhores parâmetros para uma dada semente 

In [43]:
#2º argumento -> tamanho minimo da palavra
#3º argumento -> número de vezes minima palavra aparecer
#4º argumento -> n-gramas (só funciona normal e bi-grama)
tf_idf = convert_str_to_tf_idf(x_train,3,1,1)
tokens = tf_idf.get_feature_names()


#1º argumento -> instância de td-idf
#2º argumento -> dados
data_train = convert_to_sparce_matrix(tf_idf, x_train)

data_test = convert_to_sparce_matrix(tf_idf, x_test)

penaltyTest = ['l1','l2']
multiClass = ['multinomial','ovr','auto']
cList = [0.1,0.5,1,1.5,2,2.1,2.2,2.3,2.4,2.5]
grelha={'multi_class':multiClass,'C':cList,'penalty':penaltyTest}

gSearch = GridSearchCV(LogisticRegression(solver='saga',max_iter=1000),param_grid=grelha,cv=2,verbose=1,n_jobs=3)
gSearch.fit(data_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(data_train,y_train))
print('acertos no teste: ',svm.score(data_test,y_test))
print('Parâmetros multi-classe:',par['multi_class'],'C:',par['C'],'penalty:',par['penalty'])

Fitting 2 folds for each of 60 candidates, totalling 120 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   31.0s
[Parallel(n_jobs=3)]: Done 120 out of 120 | elapsed:  3.6min finished


acertos no treino:  0.9563666666666667
acertos no teste:  0.8977
Parâmetros multi-classe: multinomial C: 2.1 penalty: l2


In [46]:
logReg = LogisticRegression(solver='saga',max_iter=1000)

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('logistic', logReg)])


param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w\w+\b'],
    'tfidf__ngram_range': [(1,1), (1,2)],
    'logistic__penalty': ['l1','l2'],
    'logistic__C': [0.5,1,1.5,2],
    'logistic__multi_class': ['multinomial','ovr']
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 128 candidates, totalling 256 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  2.6min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 16.3min
[Parallel(n_jobs=3)]: Done 256 out of 256 | elapsed: 24.1min finished


acertos no treino:  0.9885333333333334
acertos no teste:  0.9143
Parâmetros: {'logistic__C': 2, 'logistic__multi_class': 'multinomial', 'logistic__penalty': 'l2', 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


<hr style="border:2px solid black"> </hr>

## Teste com semente alteatória

In [16]:
dictionary =pickle.load(open('imdbCriticas.p','rb'))

X = dictionary.data
y = dictionary.target

y = convert_indexes_into_binary(y)

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=1/4,shuffle=True)

x_train = clear_data(x_train,False)

x_test = clear_data(x_test,False)

x_train = stemming(x_train,3)

x_test = stemming(x_test,3)

In [16]:
logReg = LogisticRegression(solver='saga',max_iter=1000)

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('logistic', logReg)])


param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'logistic__penalty': ['l1','l2'],
    'logistic__C': [1.5,1.7,1.8,1.9,2,2.1],
    'logistic__multi_class': ['multinomial','ovr']
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  9.4min
[Parallel(n_jobs=3)]: Done  96 out of  96 | elapsed: 20.7min finished


acertos no treino:  0.9863333333333333
acertos no teste:  0.914
Parâmetros: {'logistic__C': 2, 'logistic__multi_class': 'multinomial', 'logistic__penalty': 'l2', 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


In [15]:
logReg = LogisticRegression(solver='saga',max_iter=1000)

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('logistic', logReg)])


param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'logistic__penalty': ['l1','l2'],
    'logistic__C': [1.5,1.7,1.8,1.9,2,2.1],
    'logistic__multi_class': ['multinomial','ovr']
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 10.4min
[Parallel(n_jobs=3)]: Done  96 out of  96 | elapsed: 23.0min finished


acertos no treino:  0.9868333333333333
acertos no teste:  0.912
Parâmetros: {'logistic__C': 2.1, 'logistic__multi_class': 'multinomial', 'logistic__penalty': 'l2', 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


In [17]:
logReg = LogisticRegression(solver='saga',max_iter=1000)

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('logistic', logReg)])


param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'logistic__penalty': ['l1','l2'],
    'logistic__C': [1.5,1.7,1.8,1.9,2,2.1],
    'logistic__multi_class': ['multinomial','ovr']
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  9.1min
[Parallel(n_jobs=3)]: Done  96 out of  96 | elapsed: 19.6min finished


acertos no treino:  0.9866333333333334
acertos no teste:  0.9156
Parâmetros: {'logistic__C': 2.1, 'logistic__multi_class': 'multinomial', 'logistic__penalty': 'l2', 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


<hr style="border:2px solid black"> </hr>

## Testes do classificador Ridge para multi-classe 

### Recarregamento dos dados 

In [49]:
dictionary =pickle.load(open('imdbCriticas.p','rb'))

X = dictionary.data

y = dictionary.target


x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=1/4,shuffle=True,random_state=0)

x_train = clear_data(x_train,False)

x_test = clear_data(x_test,False)


x_train = stemming(x_train,3)

x_test = stemming(x_test,3)


tf_idf = convert_str_to_tf_idf(x_train,3,1,1)


data_train = convert_to_sparce_matrix(tf_idf, x_train)

data_test = convert_to_sparce_matrix(tf_idf, x_test)

### Teste para diversos valores de alpha

In [50]:
c_values =  [.1,.5,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2,5,10,15,20]


for i in range(len(c_values)):

	test_ridge(data_train,y_train,data_test,y_test,c_values[i],True,False)

Alpha: 0.1, fit_intercept: True, normalize: False
score treino: 0.9532
score teste: 0.3788
Coeficientes utilizados: 193752 

Alpha: 0.5, fit_intercept: True, normalize: False
score treino: 0.8747
score teste: 0.428
Coeficientes utilizados: 193752 

Alpha: 1, fit_intercept: True, normalize: False
score treino: 0.8141
score teste: 0.4437
Coeficientes utilizados: 193752 

Alpha: 1.1, fit_intercept: True, normalize: False
score treino: 0.8041
score teste: 0.4457
Coeficientes utilizados: 193752 

Alpha: 1.2, fit_intercept: True, normalize: False
score treino: 0.7955
score teste: 0.4482
Coeficientes utilizados: 193752 

Alpha: 1.3, fit_intercept: True, normalize: False
score treino: 0.7875
score teste: 0.4494
Coeficientes utilizados: 193752 

Alpha: 1.4, fit_intercept: True, normalize: False
score treino: 0.7794
score teste: 0.4504
Coeficientes utilizados: 193752 

Alpha: 1.5, fit_intercept: True, normalize: False
score treino: 0.7719
score teste: 0.4517
Coeficientes utilizados: 193752 

Alp

### Testes sobre os parâmetros min_df e token_pattern 

In [51]:
for sizeWord in range(1,6):

	for appearTexts in range(1,6):

		print("Tamanho da palavra:",sizeWord)

		print("Quantas vezes aparece a palavra:",appearTexts)


		tf_idf = convert_str_to_tf_idf(x_train,sizeWord,appearTexts,1)
		tokens = tf_idf.get_feature_names()
		print(len(tokens))


		data_train = convert_to_sparce_matrix(tf_idf, x_train)

		data_test = convert_to_sparce_matrix(tf_idf, x_test)


		test_ridge(data_train,y_train,data_test,y_test,5,True,False)

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 1
53673
Alpha: 5, fit_intercept: True, normalize: False
score treino: 0.6551
score teste: 0.4584
Coeficientes utilizados: 429384 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 2
53632
Alpha: 5, fit_intercept: True, normalize: False
score treino: 0.6426
score teste: 0.4406
Coeficientes utilizados: 429056 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 3
52967
Alpha: 5, fit_intercept: True, normalize: False
score treino: 0.6446
score teste: 0.4391
Coeficientes utilizados: 423736 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 4
50359
Alpha: 5, fit_intercept: True, normalize: False
score treino: 0.6478
score teste: 0.4322
Coeficientes utilizados: 402872 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 5
44277
Alpha: 5, fit_intercept: True, normalize: False
score treino: 0.6351
score teste: 0.4119
Coeficientes utilizados: 354216 

Tamanho da palavra: 2
Quantas vezes aparece a palavra: 1
30486
Alpha: 

<hr style="border:2px solid black"> </hr>

## Teste melhores parâmetros para tf-idf e ridge para uma dada semente

In [53]:
ridge = RidgeClassifier()

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('ridge', ridge)])

param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w\w+\b', r'\b\w\w\w\w\w+\b'],
    'tfidf__ngram_range': [(1,1), (1,2)],
    'ridge__alpha': [0.1,0.5,1,1.5,1.6,1.7,1.8,1.9,2,2,2.1,2.2,2.3,2.4,2.5]
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 180 candidates, totalling 360 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  2.1min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:  8.0min
[Parallel(n_jobs=3)]: Done 360 out of 360 | elapsed: 14.5min finished


acertos no treino:  0.9377333333333333
acertos no teste:  0.4829
Parâmetros: {'ridge__alpha': 2.2, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


<hr style="border:2px solid black"> </hr>

##  Testes para semente aleatória

In [15]:
dictionary =pickle.load(open('imdbCriticas.p','rb'))

X = dictionary.data

y = dictionary.target


x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=1/4,shuffle=True)

x_train = clear_data(x_train,False)

x_test = clear_data(x_test,False)


#2º argumento -> 1 - Porter Stemmer, 2 -> Lancaster Stemmer, outro -> Snowball Stemmer
x_train = stemming(x_train,3)

x_test = stemming(x_test,3)

In [20]:
ridge = RidgeClassifier()

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('ridge', ridge)])

param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w\w+\b', r'\b\w\w\w\w\w+\b'],
    'tfidf__ngram_range': [(1,1), (1,2)],
    'ridge__alpha': [0.1,0.5,1,1.5,1.6,1.7,1.8,1.9,2,2,2.1,2.2,2.3,2.4,2.5]
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 180 candidates, totalling 360 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  2.1min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:  8.0min
[Parallel(n_jobs=3)]: Done 360 out of 360 | elapsed: 14.4min finished


acertos no treino:  0.9750333333333333
acertos no teste:  0.4855
Parâmetros: {'ridge__alpha': 1.5, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


In [22]:
ridge = RidgeClassifier()

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('ridge', ridge)])

param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w\w+\b', r'\b\w\w\w\w\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'ridge__alpha': [1,1.5,1.6,1.7,1.8,1.9,2,2,2.1,2.2]
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 60 candidates, totalling 120 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  2.6min
[Parallel(n_jobs=3)]: Done 120 out of 120 | elapsed:  6.8min finished


acertos no treino:  0.9541
acertos no teste:  0.4874
Parâmetros: {'ridge__alpha': 1.9, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


In [12]:
ridge = RidgeClassifier()

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('ridge', ridge)])

param_grid = {
    'tfidf__min_df': [1,2,3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'ridge__alpha': [0,5,1,1.5,1.6,1.7,1.8,1.9,2,2,2.1,2.2]
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 96 candidates, totalling 192 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  5.1min
[Parallel(n_jobs=3)]: Done 192 out of 192 | elapsed: 18.3min finished


acertos no treino:  0.9710333333333333
acertos no teste:  0.4815
Parâmetros: {'ridge__alpha': 1.7, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


In [16]:
ridge = RidgeClassifier()

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('ridge', ridge)])

param_grid = {
    'tfidf__min_df': [1,2,3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'ridge__alpha': [0,5,1,1.5,1.6,1.7,1.8,1.9,2,2,2.1,2.2]
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 96 candidates, totalling 192 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  6.1min
[Parallel(n_jobs=3)]: Done 192 out of 192 | elapsed: 23.4min finished


acertos no treino:  0.9873333333333333
acertos no teste:  0.4794
Parâmetros: {'ridge__alpha': 1.5, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


<hr style="border:2px solid black"> </hr>

## Testes do classificador Ridge para binário

In [61]:
dictionary =pickle.load(open('imdbCriticas.p','rb'))


#os dados já vêm em strings unicode
X = dictionary.data

#valores estão baralhados
y = dictionary.target

y = convert_indexes_into_binary(y)


x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=1/4,shuffle=True,random_state=0)


#2º argumento -> true caso os dados seja uma string de bytes
x_train = clear_data(x_train,False)

x_test = clear_data(x_test,False)


#2º argumento -> 1 - Porter Stemmer, 2 -> Lancaster Stemmer, outro -> Snowball Stemmer
x_train = stemming(x_train,3)

x_test = stemming(x_test,3)

#2º argumento -> tamanho minimo da palavra
#3º argumento -> número de vezes minima palavra aparecer
#4º argumento -> n-gramas (só funciona normal e bi-grama)
tf_idf = convert_str_to_tf_idf(x_train,3,1,2)
tokens = tf_idf.get_feature_names()

### Teste para varios valores de alpha 

In [58]:
c_values =  [.1,.5,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2,5,10,15,20]


for i in range(len(c_values)):

	test_ridge(data_train,y_train,data_test,y_test,c_values[i],True,False)

Alpha: 0.1, fit_intercept: True, normalize: False
score treino: 0.9656
score teste: 0.8245
Coeficientes utilizados: 14521 

Alpha: 0.5, fit_intercept: True, normalize: False
score treino: 0.9513
score teste: 0.8519
Coeficientes utilizados: 14521 

Alpha: 1, fit_intercept: True, normalize: False
score treino: 0.9412
score teste: 0.8608
Coeficientes utilizados: 14521 

Alpha: 1.1, fit_intercept: True, normalize: False
score treino: 0.9392
score teste: 0.8624
Coeficientes utilizados: 14521 

Alpha: 1.2, fit_intercept: True, normalize: False
score treino: 0.9375
score teste: 0.8625
Coeficientes utilizados: 14521 

Alpha: 1.3, fit_intercept: True, normalize: False
score treino: 0.9359
score teste: 0.8642
Coeficientes utilizados: 14521 

Alpha: 1.4, fit_intercept: True, normalize: False
score treino: 0.9348
score teste: 0.8651
Coeficientes utilizados: 14521 

Alpha: 1.5, fit_intercept: True, normalize: False
score treino: 0.9337
score teste: 0.8658
Coeficientes utilizados: 14521 

Alpha: 1.6

In [59]:
#N-gramas de 1 (sem nada)
for sizeWord in range(1,6):

	for appearTexts in range(1,6):

		print("Tamanho da palavra:",sizeWord)

		print("Quantas vezes aparece a palavra:",appearTexts)


		tf_idf = convert_str_to_tf_idf(x_train,sizeWord,appearTexts,1)
		tokens = tf_idf.get_feature_names()
		print(len(tokens))


		data_train = convert_to_sparce_matrix(tf_idf, x_train)

		data_test = convert_to_sparce_matrix(tf_idf, x_test)


		test_ridge(data_train,y_train,data_test,y_test,2,True,False)

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 1
53673
Alpha: 2, fit_intercept: True, normalize: False
score treino: 0.9542
score teste: 0.8987
Coeficientes utilizados: 53673 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 2
53632
Alpha: 2, fit_intercept: True, normalize: False
score treino: 0.9533
score teste: 0.8964
Coeficientes utilizados: 53632 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 3
52967
Alpha: 2, fit_intercept: True, normalize: False
score treino: 0.9527
score teste: 0.8951
Coeficientes utilizados: 52967 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 4
50359
Alpha: 2, fit_intercept: True, normalize: False
score treino: 0.951
score teste: 0.8886
Coeficientes utilizados: 50359 

Tamanho da palavra: 1
Quantas vezes aparece a palavra: 5
44277
Alpha: 2, fit_intercept: True, normalize: False
score treino: 0.9406
score teste: 0.8699
Coeficientes utilizados: 44277 

Tamanho da palavra: 2
Quantas vezes aparece a palavra: 1
30486
Alpha: 2, fit

In [62]:
alphaList = [0.1,0.5,1,1.1,1.2,1.3,1.4,1.5,2,2.5]
fitList = [True,False]
normList = [True,False]
#cList = [0.1,0.2]
grelha={'alpha':alphaList,'fit_intercept':fitList,'normalize':normList}

gSearch = GridSearchCV(RidgeClassifier(),param_grid=grelha,cv=5,verbose=1,n_jobs=3)
gSearch.fit(data_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(data_train,y_train))
print('acertos no teste: ',svm.score(data_test,y_test))
print('Parâmetros: alpha:',par['alpha'],'fit_intercept:',par['fit_intercept'],'normalize:',par['normalize'])

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    3.8s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:   11.0s
[Parallel(n_jobs=3)]: Done 200 out of 200 | elapsed:   11.3s finished


acertos no treino:  0.9292333333333334
acertos no teste:  0.8664
Parâmetros: alpha: 2 fit_intercept: True normalize: False


<hr style="border:2px solid black"> </hr>

## Teste para uma determinada semente 

In [63]:
ridge = RidgeClassifier()

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('ridge', ridge)])

param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w\w+\b', r'\b\w\w\w\w\w+\b'],
    'tfidf__ngram_range': [(1,1), (1,2)],
    'ridge__alpha': [0.1,0.5,1,1.5,1.6,1.7,1.8,1.9,2,2,2.1,2.2,2.3,2.4,2.5]
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 180 candidates, totalling 360 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  1.6min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:  7.0min
[Parallel(n_jobs=3)]: Done 360 out of 360 | elapsed: 12.8min finished


acertos no treino:  0.9965333333333334
acertos no teste:  0.9175
Parâmetros: {'ridge__alpha': 1, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


<hr style="border:2px solid black"> </hr>

## Teste para semente aleatória 

In [15]:
dictionary =pickle.load(open('imdbCriticas.p','rb'))

X = dictionary.data

y = dictionary.target

y = convert_indexes_into_binary(y)

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=1/4,shuffle=True)

x_train = clear_data(x_train,False)

x_test = clear_data(x_test,False)

x_train = stemming(x_train,3)

x_test = stemming(x_test,3)

In [30]:
ridge = RidgeClassifier()

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('ridge', ridge)])

param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w\w+\b', r'\b\w\w\w\w\w+\b'],
    'tfidf__ngram_range': [(1,1), (1,2)],
    'ridge__alpha': [0.1,0.5,1,1.5,1.6,1.7,1.8,1.9,2,2,2.1,2.2,2.3,2.4,2.5]
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 180 candidates, totalling 360 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  1.5min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:  6.5min
[Parallel(n_jobs=3)]: Done 360 out of 360 | elapsed: 12.0min finished


acertos no treino:  0.9963666666666666
acertos no teste:  0.9162
Parâmetros: {'ridge__alpha': 1, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


In [32]:
ridge = RidgeClassifier()

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('ridge', ridge)])

param_grid = {
    'tfidf__min_df': [1,2,3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w\w+\b', r'\b\w\w\w\w\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'ridge__alpha': [0.5,1,1.5,1.9,2,2.1]
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 72 candidates, totalling 144 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  2.4min
[Parallel(n_jobs=3)]: Done 144 out of 144 | elapsed:  7.5min finished


acertos no treino:  0.9998666666666667
acertos no teste:  0.9134
Parâmetros: {'ridge__alpha': 0.5, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


In [34]:
ridge = RidgeClassifier()

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('ridge', ridge)])

param_grid = {
    'tfidf__min_df': [1,2,3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w\w+\b', r'\b\w\w\w\w\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'ridge__alpha': [0.5,1,1.5,1.9,2,2.1]
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 72 candidates, totalling 144 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  2.4min
[Parallel(n_jobs=3)]: Done 144 out of 144 | elapsed:  7.5min finished


acertos no treino:  0.9999666666666667
acertos no teste:  0.9152
Parâmetros: {'ridge__alpha': 0.5, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


In [18]:
ridge = RidgeClassifier()

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('ridge', ridge)])

param_grid = {
    'tfidf__min_df': [1,2,3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'ridge__alpha': [0.5,1,1.5,1.9,2,2.1]
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  3.3min
[Parallel(n_jobs=3)]: Done  96 out of  96 | elapsed:  7.1min finished


acertos no treino:  0.997
acertos no teste:  0.9165
Parâmetros: {'ridge__alpha': 1, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


In [13]:
ridge = RidgeClassifier(max_iter=1000)

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('ridge', ridge)])

param_grid = {
    'tfidf__min_df': [1,2,3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'ridge__alpha': [0.5,1,1.5,1.9,2,2.1]
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  3.5min
[Parallel(n_jobs=3)]: Done  96 out of  96 | elapsed:  7.2min finished


acertos no treino:  0.9999666666666667
acertos no teste:  0.9142
Parâmetros: {'ridge__alpha': 0.5, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


In [16]:
ridge = RidgeClassifier(max_iter=1000)

tf_idf = TfidfVectorizer()

pipe = Pipeline([('tfidf', tf_idf), ('ridge', ridge)])

param_grid = {
    'tfidf__min_df': [1,2,3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'ridge__alpha': [0.5,1,1.5,1.9,2,2.1]
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  3.6min
[Parallel(n_jobs=3)]: Done  96 out of  96 | elapsed:  7.5min finished


acertos no treino:  0.9999333333333333
acertos no teste:  0.9158
Parâmetros: {'ridge__alpha': 0.5, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


# Utilização de PCA no pré-processamento 

In [13]:
dictionary = pickle.load(open('imdbCriticas.p','rb'))

X = dictionary.data

y = dictionary.target

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=1/4,shuffle=True,random_state=0)


x_train = clear_data(x_train,False)

x_test = clear_data(x_test,False)


x_train = stemming(x_train,3)

x_test = stemming(x_test,3)


tf_idf = convert_str_to_tf_idf(x_train,3,1,1)


data_train = convert_to_sparce_matrix(tf_idf, x_train)

data_test = convert_to_sparce_matrix(tf_idf, x_test)


logReg = LogisticRegression(penalty='l2',solver='saga',max_iter=1000,C=2,tol=1e-3,multi_class='ovr')
logReg.fit(data_train,y_train)

print(logReg.score(data_train,y_train))
print(logReg.score(data_test,y_test))

0.7142666666666667
0.4579


In [18]:
dictionary = pickle.load(open('imdbCriticas.p','rb'))

X = dictionary.data

y = dictionary.target

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=1/4,shuffle=True,random_state=0)


x_train = clear_data(x_train,False)

x_test = clear_data(x_test,False)


x_train = stemming(x_train,3)

x_test = stemming(x_test,3)


tf_idf = convert_str_to_tf_idf(x_train,3,1,1)


data_train = convert_to_sparce_matrix(tf_idf, x_train)

data_test = convert_to_sparce_matrix(tf_idf, x_test)

# utilizar 5 componentes principais
svd = TruncatedSVD(n_components=5, n_iter=7).fit(data_train)
data_train = svd.transform(data_train)
data_test = svd.transform(data_test)


logReg = LogisticRegression(penalty='l2',solver='saga',max_iter=1000,C=2,tol=1e-3,multi_class='ovr')
logReg.fit(data_train,y_train)

print(logReg.score(data_train,y_train))
print(logReg.score(data_test,y_test))

0.262
0.2607


<hr style="border:2px solid black"> </hr>

### Teste para classificação multi-classe 

In [16]:
tf_idf = TfidfVectorizer()

logReg = LogisticRegression(solver='saga',max_iter=1000)

svd = TruncatedSVD(n_components=5, n_iter=100)

pipe = Pipeline([('tfidf', tf_idf), ('svd', svd), ('logistic', logReg)])

param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b', r'\b\w\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'svd__n_components': [5,10,20,50,100],
    'logistic__penalty': ['l1'],
    'logistic__C': [0.5,1,1.5,2],
    'logistic__multi_class': ['multinomial','ovr']
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 160 candidates, totalling 320 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 22.7min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 106.9min
[Parallel(n_jobs=3)]: Done 320 out of 320 | elapsed: 181.1min finished


acertos no treino:  0.4429
acertos no teste:  0.4339
Parâmetros: {'logistic__C': 2, 'logistic__multi_class': 'multinomial', 'logistic__penalty': 'l1', 'svd__n_components': 100, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


In [12]:
dictionary = pickle.load(open('imdbCriticas.p','rb'))

X = dictionary.data

y = dictionary.target

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=1/4,shuffle=True,random_state=0)



x_train = clear_data(x_train,False)

x_test = clear_data(x_test,False)

x_train = stemming(x_train,3)

x_test = stemming(x_test,3)

tf_idf = TfidfVectorizer()

logReg = LogisticRegression(solver='saga',max_iter=1000)

svd = TruncatedSVD(n_iter=1000)

pipe = Pipeline([('tfidf', tf_idf), ('svd', svd), ('logistic', logReg)])


param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'svd__n_components': [110],
    'logistic__penalty': ['l1'],
    'logistic__C': [2],
    'logistic__multi_class': ['multinomial','ovr']
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   8 out of   8 | elapsed: 114.7min finished


acertos no treino:  0.44483333333333336
acertos no teste:  0.4346
Parâmetros: {'logistic__C': 2, 'logistic__multi_class': 'multinomial', 'logistic__penalty': 'l1', 'svd__n_components': 110, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}


### Teste para classificação binária 

In [12]:
dictionary = pickle.load(open('imdbCriticas.p','rb'))

X = dictionary.data

y = dictionary.target

y = convert_indexes_into_binary(y)

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=1/4,shuffle=True,random_state=0)

x_train = clear_data(x_train,False)

x_test = clear_data(x_test,False)

x_train = stemming(x_train,3)

x_test = stemming(x_test,3)

tf_idf = TfidfVectorizer()

logReg = LogisticRegression(solver='saga',max_iter=1000)

svd = TruncatedSVD(n_components=5, n_iter=100)

pipe = Pipeline([('tfidf', tf_idf), ('svd', svd), ('logistic', logReg)])

param_grid = {
    'tfidf__min_df': [3, 5],
    'tfidf__token_pattern': [r'\b\w+\b'],
    'tfidf__ngram_range': [(1,2)],
    'svd__n_components': [5,10,20,50],
    'logistic__penalty': ['l1'],
    'logistic__C': [0.5,1,1.5,2],
    'logistic__multi_class': ['multinomial','ovr']
}

gSearch = GridSearchCV(pipe,param_grid=param_grid,cv=2,verbose=1,n_jobs=3)
gSearch.fit(x_train,y_train)

svm = gSearch.best_estimator_
par = gSearch.best_params_

print('acertos no treino: ',svm.score(x_train,y_train))
print('acertos no teste: ',svm.score(x_test,y_test))
print('Parâmetros:',par)

Fitting 2 folds for each of 64 candidates, totalling 128 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 17.4min
[Parallel(n_jobs=3)]: Done 128 out of 128 | elapsed: 51.3min finished


acertos no treino:  0.8554333333333334
acertos no teste:  0.8568
Parâmetros: {'logistic__C': 2, 'logistic__multi_class': 'ovr', 'logistic__penalty': 'l1', 'svd__n_components': 50, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2), 'tfidf__token_pattern': '\\b\\w+\\b'}
